In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier

In [5]:
df = pd.read_csv(r"C:\Users\Admin\OneDrive\Documents\train.csv (2)\train.csv")
unseen_data = pd.read_csv(r"C:\Users\Admin\OneDrive\Documents\testsf.csv\test.csv")
df.head()

,Dates,Category,Descript,DayOfWeek,PdDistrict,Resolution,Address,X,Y
0,2015-05-13 23:53:00,WARRANTS,WARRANT ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",OAK ST / LAGUNA ST,-122.425892,37.774599
1,2015-05-13 23:53:00,OTHER OFFENSES,TRAFFIC VIOLATION ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",OAK ST / LAGUNA ST,-122.425892,37.774599
2,2015-05-13 23:33:00,OTHER OFFENSES,TRAFFIC VIOLATION ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",VANNESS AV / GREENWICH ST,-122.424363,37.800414
3,2015-05-13 23:30:00,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Wednesday,NORTHERN,NONE,1500 Block of LOMBARD ST,-122.426995,37.800873
4,2015-05-13 23:30:00,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Wednesday,PARK,NONE,100 Block of BRODERICK ST,-122.438738,37.771541


In [16]:
lat = df['X'].nunique()
long = df['Y'].nunique()
print(lat)
print(long)

34243
34243


Next iteration , figure out a way to incorprate the lat and long without too much cardinality and as a categorical feat obvs .
ex. round to 100s and 1000s place so similar neighbouroods are grouped 
check how many unqie paLat and long pairs there are , maybe less 
- false there would be at least 34243 pairs

In [10]:
df['Descript'].value_counts()

Descript
GRAND THEFT FROM LOCKED AUTO                              60022
LOST PROPERTY                                             31729
BATTERY                                                   27441
STOLEN AUTOMOBILE                                         26897
DRIVERS LICENSE, SUSPENDED OR REVOKED                     26839
                                                          ...  
ENCOURAGE MINOR TO USE CONTROLLED SUBSTANCE                   1
BIGAMY, INCEST, AND THE CRIME AGAINST NATURE (GENERAL)        1
DISTURBANCE OF NON-RELIGIOUS, NON-POLITICAL ASSEMBLY          1
CABLE TV CONNECTION OR DECODING DEVICE, UNAUTHORIZED          1
EMBEZZLEMENT, GRAND THEFT PUBLIC/PRIVATE OFFICIAL             1
Name: count, Length: 879, dtype: int64

In [11]:
# limit column cardinality

top200Des = df['Descript'].value_counts()[:200].index

df['Descript'] = df['Descript'].apply(lambda x :x if x in top200Des else 'other' )  # may need to put this as a function in model pipeline for test set

In [17]:
x = df.drop(['Category', 'Address', 'Dates', 'X', 'Y'], axis = 1)
y = df['Category']

You want to preserve the cyclical nature of your inputs. 
One approach is to cut the datetime variable into four variables: year, month, day, and hour.
Then, decompose each of these (except for year) variables in two.

You create a sine and a cosine facet of each of these three variables (i.e., month, day, hour), which will retain the fact that hour 24 is closer to hour 0 than to hour 21, 
and that month 12 is closer to month 1 than to month 10.

from https://stats.stackexchange.com/questions/311494/best-practice-for-encoding-datetime-in-machine-learning 
check link for additional links given by answerer

In [18]:

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.15, random_state = 1)

In [27]:
x_test.shape

(131708, 4)

In [31]:
x_train.columns

Index(['Descript', 'DayOfWeek', 'PdDistrict', 'Resolution'], dtype='object')

In [30]:
x_train['Resolution'].value_counts()

Resolution
NONE                                      447755
ARREST, BOOKED                            175250
ARREST, CITED                              65679
LOCATED                                    14563
PSYCHOPATHIC CASE                          12335
UNFOUNDED                                   8170
JUVENILE BOOKED                             4716
COMPLAINANT REFUSES TO PROSECUTE            3359
DISTRICT ATTORNEY REFUSES TO PROSECUTE      3343
NOT PROSECUTED                              3125
JUVENILE CITED                              2842
PROSECUTED BY OUTSIDE AGENCY                2131
EXCEPTIONAL CLEARANCE                       1315
JUVENILE ADMONISHED                         1215
JUVENILE DIVERTED                            301
CLEARED-CONTACT JUVENILE FOR MORE INFO       196
PROSECUTED FOR LESSER OFFENSE                 46
Name: count, dtype: int64

In [9]:
model = XGBClassifier()

In [38]:
ohc = OneHotEncoder()

In [36]:
# Encode target labels (y_train)

# Reshape and encode the target labels
y_train_encoded = ohc.fit_transform(y_train.values.reshape(-1, 1)).toarray()

In [32]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer


ohc = OneHotEncoder()
cat = ['Descript', 'DayOfWeek','PdDistrict', 'Resolution']

column_transformer = ColumnTransformer(
    transformers=[
        ('cat_transform', ohc, cat)
    ]
)

pipe = Pipeline(steps=[('cat_transform', column_transformer), 
                ('mod', model) ])



In [39]:
pipe.fit(x_train, y_train_encoded)

Pipeline(steps=[('cat_transform',
                 ColumnTransformer(transformers=[('cat_transform',
                                                  OneHotEncoder(),
                                                  ['Descript', 'DayOfWeek',
                                                   'PdDistrict',
                                                   'Resolution'])])),
                ('mod',
                 XGBClassifier(base_score=None, booster=None, callbacks=None,
                               colsample_bylevel=None, colsample_bynode=None,
                               colsample_bytree=None,
                               early_stopping_rounds=None,
                               enable_categorical=False, eval_metric=N...
                               feature_types=None, gamma=None, gpu_id=None,
                               grow_policy=None, importance_type=None,
                               interaction_constraints=None, learning_rate=None,
                               max_bin=None, max_cat_threshold=None,
                               max_cat_to_onehot=None, max_delta_step=None,
                               max_depth=None, max_leaves=None,
                               min_child_weight=None, missing=nan,
                               monotone_constraints=None, n_estimators=100,
                               n_jobs=None, num_parallel_tree=None,
                               predictor=None, random_state=None, ...))])

In [42]:
# Fit the OneHotEncoder on training labels (y_train)
# need to fit OHC on training y before transforming y_test (ie. fit_transfrom (y_train) wont work )
ohc.fit(y_train.values.reshape(-1, 1))


OneHotEncoder()

In [43]:
# Encode the test labels (y_test)
y_test_encoded = ohc.transform(y_test.values.reshape(-1, 1)).toarray()

# predict and check accuracy 
y_pred = pipe.predict(x_test)
accuracy_score(y_pred, y_test_encoded)

0.9405047529383181